## Visualize your performance metrics on Wandb

### Required libraries: wandb, nbformat

### Step 1: Visit https://wandb.ai/authorize

### Step 2: login with your account or with Google

### Step 3: Copy API key

### Step 4: login Wandb 

- In VSCode Terminal, run python3 (or python)

- import wandb

- wandb.login()

- Enter your choice: 2

- paste the copied API key


### Step 5: exit pyhton environment from Terminal

### Step 6: run your script with wandb.init() and then wandb.log, and wandb.finish() to end wandb run at the end.


In [1]:

import networkx as nx
import scipy.sparse as sp
import psutil, time, os, gc, statistics, warnings
import numpy as np
import pandas as pd
from codecarbon import EmissionsTracker
from collections import defaultdict
import wandb

warnings.filterwarnings('ignore')


In [ ]:

# Function to get memory usage
def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024 / 1024  # Memory in MB

# Function to get CPU usage
def get_cpu_usage():
    return psutil.cpu_percent(interval=1)

# Function to run analysis with multiple iterations
def run_analysis_with_repeats(analysis_func, file_path, method_name, num_runs=5):
    results = []
    emissions = []
    cpu_usages = []
    
    # Initialize wandb run for this method
    wandb.init(project="network_analysis", name=f"{method_name}_run", reinit=True)
    
    for run_idx in range(num_runs):
        gc.collect()  # Clear memory before each run
        tracker = EmissionsTracker()
        tracker.start()
        start_cpu = get_cpu_usage()
        
        result = analysis_func(file_path)
        
        end_cpu = get_cpu_usage()
        emission = tracker.stop()
        emissions.append(emission)
        results.append(result)
        cpu_usages.append((start_cpu + end_cpu) / 2)  # Average CPU usage during run
        
        # Log per-run metrics to wandb
        wandb.log({
            "Run": run_idx + 1,
            "Time (s)": result["Time (s)"],
            "Memory (MB)": result["Memory (MB)"],
            "Emissions (kgCO2)": emission,
            "CPU Usage (%)": cpu_usages[-1]
        })
    
    # Compute statistics
    metrics = {
        "Method": results[0]["Method"],
        "Nodes": results[0]["Nodes"],
        "Edges": results[0]["Edges"],
        "Avg Degree": results[0]["Avg Degree"],
        "Density": results[0]["Density"],
        "Longest Path": results[0]["Longest Path"],
        "Longest Path Nodes": results[0]["Longest Path Nodes"],
        "Shortest Path": results[0]["Shortest Path"],
        "Shortest Path Nodes": results[0]["Shortest Path Nodes"],
        "Time (s)": statistics.mean([r["Time (s)"] for r in results]),
        "Time Std (s)": statistics.stdev([r["Time (s)"] for r in results]) if num_runs > 1 else 0,
        "Memory (MB)": statistics.mean([r["Memory (MB)"] for r in results]),
        "Memory Std (MB)": statistics.stdev([r["Memory (MB)"] for r in results]) if num_runs > 1 else 0,
        "Emissions (kgCO2)": statistics.mean(emissions),
        "Emissions Std (kgCO2)": statistics.stdev(emissions) if num_runs > 1 else 0,
        "CPU Usage (%)": statistics.mean(cpu_usages),
        "CPU Usage Std (%)": statistics.stdev(cpu_usages) if num_runs > 1 else 0
    }
    
    # Log aggregated metrics to wandb
    wandb.log({
        "Avg Time (s)": metrics["Time (s)"],
        "Avg Memory (MB)": metrics["Memory (MB)"],
        "Avg Emissions (kgCO2)": metrics["Emissions (kgCO2)"],
        "Avg CPU Usage (%)": metrics["CPU Usage (%)"]
    })
    
    # Log top 20 nodes as a table
    top_nodes_df = pd.DataFrame(results[0]["Top 20 Nodes"])
    wandb.log({"Top 20 Nodes": wandb.Table(dataframe=top_nodes_df)})
    
    wandb.finish()  # End the wandb run
    return metrics, results[0]["Top 20 Nodes"]

# Naive implementation using adjacency list
def analyze_naive(file_path):
    start_time = time.time()
    start_memory = get_memory_usage()
    
    # Build adjacency list
    graph = defaultdict(set)
    with open(file_path, 'r') as f:
        for line in f:
            n1, n2 = map(int, line.strip().split())
            graph[n1].add(n2)
            graph[n2].add(n1)
    
    num_nodes = len(graph)
    num_edges = sum(len(neighbors) for neighbors in graph.values()) // 2
    avg_degree = sum(len(neighbors) for neighbors in graph.values()) / num_nodes if num_nodes > 0 else 0
    density = (2 * num_edges) / (num_nodes * (num_nodes - 1)) if num_nodes > 1 else 0
    
    # Simple BFS for shortest path and tracking nodes
    def bfs_shortest_path(start, end):
        if start == end:
            return 0, [start]
        visited = {start}
        queue = [(start, [start])]
        while queue:
            node, path = queue.pop(0)
            for neighbor in graph[node]:
                if neighbor not in visited:
                    new_path = path + [neighbor]
                    if neighbor == end:
                        return len(new_path) - 1, new_path
                    visited.add(neighbor)
                    queue.append((neighbor, new_path))
        return float('inf'), []
    
    # Find approximate longest and shortest paths
    max_path = 0
    min_path = float('inf')
    max_path_nodes = []
    min_path_nodes = []
    sample_nodes = list(graph.keys())[:min(1000, num_nodes)]  # Sample for efficiency
    for i, start in enumerate(sample_nodes):
        for end in sample_nodes[i+1:]:
            path_len, path_nodes = bfs_shortest_path(start, end)
            if path_len != float('inf'):
                if path_len > max_path:
                    max_path = path_len
                    max_path_nodes = path_nodes
                if path_len > 0 and path_len < min_path:
                    min_path = path_len
                    min_path_nodes = path_nodes
    
    # Find top 20 nodes by degree
    degrees = [(node, len(neighbors)) for node, neighbors in graph.items()]
    top_nodes = sorted(degrees, key=lambda x: x[1], reverse=True)[:20]
    top_nodes_dict = [{"Node": node, "Degree": degree} for node, degree in top_nodes]
    
    exec_time = time.time() - start_time
    memory_used = get_memory_usage() - start_memory
    
    print(f"Nodes: {num_nodes}")
    print(f"Edges: {num_edges}")
    print(f"Average Degree: {avg_degree:,.2f}")
    print(f"Density: {density:,.6f}")
    print(f"Longest Path: {max_path}")
    print(f"Longest Path Nodes: {max_path_nodes}")
    print(f"Shortest Path: {min_path if min_path != float('inf') else 'N/A'}")
    print(f"Shortest Path Nodes: {min_path_nodes if min_path != float('inf') else 'N/A'}")
    print(f"Execution Time: {exec_time:,.2f} seconds")
    print(f"Memory Used: {memory_used:,.2f} MB")
    print("Top 20 Most Connected Nodes:")
    for node in top_nodes_dict:
        print(f"  Node {node['Node']}: Degree {node['Degree']}")
    
    return {
        "Method": "Naive (Adjacency List)",
        "Nodes": num_nodes,
        "Edges": num_edges,
        "Avg Degree": avg_degree,
        "Density": density,
        "Longest Path": max_path,
        "Longest Path Nodes": max_path_nodes,
        "Shortest Path": min_path if min_path != float('inf') else 0,
        "Shortest Path Nodes": min_path_nodes if min_path != float('inf') else [],
        "Time (s)": exec_time,
        "Memory (MB)": memory_used,
        "Top 20 Nodes": top_nodes_dict
    }

# Function to analyze full graph with NetworkX
def analyze_networkx(file_path):
    start_time = time.time() 
    start_memory = get_memory_usage()
    
    G = nx.read_edgelist(file_path, nodetype=int, create_using=nx.Graph())
    
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    avg_degree = sum(dict(G.degree()).values()) / num_nodes
    density = (2 * num_edges) / (num_nodes * (num_nodes - 1)) if num_nodes > 1 else 0
    
    # Find longest and shortest paths with nodes
    longest_path = 0
    shortest_path = float('inf')
    longest_path_nodes = []
    shortest_path_nodes = []
    if num_nodes > 1:
        for path in nx.all_pairs_shortest_path(G):
            path_len = len(path[1][list(path[1].keys())[-1]]) - 1
            if path_len > longest_path:
                longest_path = path_len
                longest_path_nodes = path[1][list(path[1].keys())[-1]]
            if path_len > 0 and path_len < shortest_path:
                shortest_path = path_len
                shortest_path_nodes = path[1][list(path[1].keys())[-1]]
    
    # Find top 20 nodes by degree
    degrees = G.degree()
    top_nodes = sorted(degrees, key=lambda x: x[1], reverse=True)[:20]
    top_nodes_dict = [{"Node": node, "Degree": degree} for node, degree in top_nodes]
    
    exec_time = time.time() - start_time
    memory_used = get_memory_usage() - start_memory
    
    print(f"Nodes: {num_nodes}")
    print(f"Edges: {num_edges}")
    print(f"Average Degree: {avg_degree:,.2f}")
    print(f"Density: {density:,.6f}")
    print(f"Longest Path: {longest_path}")
    print(f"Longest Path Nodes: {longest_path_nodes}")
    print(f"Shortest Path: {shortest_path if shortest_path != float('inf') else 'N/A'}")
    print(f"Shortest Path Nodes: {shortest_path_nodes if shortest_path != float('inf') else 'N/A'}")
    print(f"Execution Time: {exec_time:,.2f} seconds")
    print(f"Memory Used: {memory_used:,.2f} MB")
    print("Top 20 Most Connected Nodes:")
    for node in top_nodes_dict:
        print(f"  Node {node['Node']}: Degree {node['Degree']}")
    
    return {
        "Method": "NetworkX (Full)",
        "Nodes": num_nodes,
        "Edges": num_edges,
        "Avg Degree": avg_degree,
        "Density": density,
        "Longest Path": longest_path,
        "Longest Path Nodes": longest_path_nodes,
        "Shortest Path": shortest_path,
        "Shortest Path Nodes": shortest_path_nodes,
        "Time (s)": exec_time,
        "Memory (MB)": memory_used,
        "Top 20 Nodes": top_nodes_dict
    }

# SciPy sparse matrix analysis
def analyze_scipy_sparse(file_path):
    start_time = time.time()   
    start_memory = get_memory_usage()
    
    # Load edge list with pandas for better performance
    edges_df = pd.read_csv(file_path, sep=' ', header=None, dtype=np.int32, engine='c')
    edges = edges_df.to_numpy()
    
    # Vectorized node mapping
    nodes, inverse_indices = np.unique(edges, return_inverse=True)
    num_nodes = len(nodes)
    edge_indices = inverse_indices.reshape(edges.shape)  # Shape: (m, 2)
    
    # Create row and column arrays for symmetric adjacency matrix
    rows = np.concatenate([edge_indices[:, 0], edge_indices[:, 1]])
    cols = np.concatenate([edge_indices[:, 1], edge_indices[:, 0]])
    data = np.ones(len(rows), dtype=np.int32)
    
    adj_matrix = sp.csr_matrix((data, (rows, cols)), shape=(num_nodes, num_nodes))
    
    num_edges = adj_matrix.nnz // 2
    degrees = np.array(adj_matrix.sum(axis=1)).flatten()
    avg_degree = degrees.mean()
    density = (2 * num_edges) / (num_nodes * (num_nodes - 1)) if num_nodes > 1 else 0
    
    # Convert to NetworkX for path calculations
    G = nx.from_scipy_sparse_array(adj_matrix)
    longest_path = 0
    shortest_path = float('inf')
    longest_path_nodes = []
    shortest_path_nodes = []
    if num_nodes > 1:
        for path in nx.all_pairs_shortest_path(G):
            path_len = len(path[1][list(path[1].keys())[-1]]) - 1
            if path_len > longest_path:
                longest_path = path_len
                longest_path_nodes = path[1][list(path[1].keys())[-1]]
            if path_len > 0 and path_len < shortest_path:
                shortest_path = path_len
                shortest_path_nodes = path[1][list(path[1].keys())[-1]]
    
    # Find top 20 nodes by degree
    top_indices = np.argpartition(degrees, -20)[-20:]
    top_degrees = degrees[top_indices]
    top_nodes = [(nodes[i], degrees[i]) for i in top_indices]
    top_nodes = sorted(top_nodes, key=lambda x: x[1], reverse=True)[:20]
    top_nodes_dict = [{"Node": node, "Degree": degree} for node, degree in top_nodes]
    
    exec_time = time.time() - start_time
    memory_used = get_memory_usage() - start_memory
    
    print(f"Nodes: {num_nodes}")
    print(f"Edges: {num_edges}")
    print(f"Average Degree: {avg_degree:,.2f}")
    print(f"Density: {density:,.6f}")
    print(f"Longest Path: {longest_path}")
    print(f"Longest Path Nodes: {longest_path_nodes}")
    print(f"Shortest Path: {shortest_path if shortest_path != float('inf') else 'N/A'}")
    print(f"Shortest Path Nodes: {shortest_path_nodes if shortest_path != float('inf') else 'N/A'}")
    print(f"Execution Time: {exec_time:,.2f} seconds")
    print(f"Memory Used: {memory_used:,.2f} MB")
    print("Top 20 Most Connected Nodes:")
    for node in top_nodes_dict:
        print(f"  Node {node['Node']}: Degree {node['Degree']}")
    
    return {
        "Method": "SciPy Sparse (Full)",
        "Nodes": num_nodes,
        "Edges": num_edges,
        "Avg Degree": avg_degree,
        "Density": density,
        "Longest Path": longest_path,
        "Longest Path Nodes": longest_path_nodes,
        "Shortest Path": shortest_path,
        "Shortest Path Nodes": shortest_path_nodes,
        "Time (s)": exec_time,
        "Memory (MB)": memory_used,
        "Top 20 Nodes": top_nodes_dict
    }

# Function to run all analyses
def run_all_analyses(file_path, num_runs):
    methods = [
        ("Naive", analyze_naive),
        ("NetworkX", analyze_networkx),
        ("SciPy Sparse", analyze_scipy_sparse)
    ]
    results = []
    top_nodes_dfs = []
    
    for method_name, method_func in methods:
        print(f"\nRunning {method_name} analysis ({num_runs} runs)")
        metrics, top_nodes = run_analysis_with_repeats(method_func, file_path, method_name, num_runs)
        results.append(metrics)
        top_nodes_df = pd.DataFrame(top_nodes)
        top_nodes_df['Method'] = f"{method_name} (Full)"
        top_nodes_dfs.append(top_nodes_df)
    
    return results, top_nodes_dfs


In [3]:

# Main execution
def main():
    # Initialize wandb project
    # wandb.login()  # Ensure you are logged in or set WANDB_API_KEY environment variable, complete this step before running in terminal
    
    
    file_path = "data/facebook_combined.txt"
    
    if not os.path.exists(file_path):
        print(f"Error: {file_path} not found. Please download from http://snap.stanford.edu/data/egnets-Facebook.html")
        wandb.finish()
        return
    
    num_runs = 5
    results, top_nodes_dfs = run_all_analyses(file_path, num_runs)
    
    wandb.init(project="network_analysis", name="main_comparison")
    
    # Combine results
    print("\nTable: Performance Comparison (Averaged over runs)")
    df = pd.DataFrame(results)
    print(df.round(6))
    df.to_csv("01-03-graph_analysis_comparison.csv", index=False)
    print("\nResults saved to 01-03-graph_analysis_comparison.csv")
    
    # Log comparison table to wandb
    wandb.log({"Performance Comparison": wandb.Table(dataframe=df)})
    
    # Save top nodes to CSV and log to wandb
    top_nodes_combined = pd.concat(top_nodes_dfs, ignore_index=True)
    top_nodes_combined.to_csv("01-03-top_20_nodes.csv", index=False)
    print("Top 20 nodes saved to 01-03-top_20_nodes.csv")
    wandb.log({"Top 20 Nodes Combined": wandb.Table(dataframe=top_nodes_combined)})
    
    wandb.finish()

if __name__ == "__main__":
    main()



Running Naive analysis (5 runs)


wandb: Currently logged in as: jl-liam-gao to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


[codecarbon INFO @ 08:19:30] Codecarbon is taking the configuration from the local file /Users/jgao/github/ECRI-RCDS-Hackathon/.codecarbon.config
[codecarbon WARNING @ 08:19:30] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon WARNING @ 08:19:30] Error while trying to count physical CPUs: [Errno 2] No such file or directory: 'lscpu'. Defaulting to 1.
[codecarbon INFO @ 08:19:30] [setup] RAM Tracking...
[codecarbon INFO @ 08:19:30] [setup] CPU Tracking...
[codecarbon WARNING @ 08:19:30] We saw that you have a Apple M4 but we don't know it. Please contact us.
[codecarbon DEBUG @ 08:19:30] is_psutil_available() : psutil.cpu_times().nice is too small : 0.0 !
[codecarbon DEBUG @ 08:19:30] Not using PowerGadget, an exception occurred while instantiating IntelPowerGadget : Intel Power Gadget executable not found on darwin
[codecarbon DEBUG @ 08:19:30] Not using the RAPL interface, an exception occurred while instantiating IntelRAPL : Platform not supported by 

Nodes: 4039
Edges: 88234
Average Degree: 43.69
Density: 0.010820
Longest Path: 3
Longest Path Nodes: [1, 0, 34, 348]
Shortest Path: 1
Shortest Path Nodes: [0, 1]
Execution Time: 44.10 seconds
Memory Used: -73.17 MB
Top 20 Most Connected Nodes:
  Node 107: Degree 1045
  Node 1684: Degree 792
  Node 1912: Degree 755
  Node 3437: Degree 547
  Node 0: Degree 347
  Node 2543: Degree 294
  Node 2347: Degree 291
  Node 1888: Degree 254
  Node 1800: Degree 245
  Node 1663: Degree 235
  Node 1352: Degree 234
  Node 2266: Degree 234
  Node 483: Degree 231
  Node 348: Degree 229
  Node 1730: Degree 226
  Node 1985: Degree 224
  Node 1941: Degree 223
  Node 2233: Degree 222
  Node 2142: Degree 221
  Node 1431: Degree 220


[codecarbon DEBUG @ 08:20:20] RAM power estimation: 3.00W for 16.00GB
[codecarbon INFO @ 08:20:20] Energy consumed for RAM : 0.000038 kWh. RAM Power : 3.0 W
[codecarbon DEBUG @ 08:20:20] Done measure for RAM - measurement time: 0.0023 s - last call 1.00 s
[codecarbon INFO @ 08:20:20] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 08:20:20] Energy consumed for All CPU : 0.000539 kWh
[codecarbon DEBUG @ 08:20:20] Done measure for CPU - measurement time: 0.0007 s - last call 1.01 s
[codecarbon INFO @ 08:20:20] 0.000577 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 08:20:20] last_duration=1.004513666004641
------------------------
[codecarbon DEBUG @ 08:20:20] We apply an energy mix of 238 g.CO2eq/kWh for United Kingdom
[codecarbon DEBUG @ 08:20:20] EmissionsData(timestamp='2025-07-12T08:20:20', project_name='codecarbon', run_id='None', experiment_id='d960f128-b0b6-4eb2-a1e1-d2bbc81bbbe0', duration=46.11942520798766, emissio

...

[codecarbon DEBUG @ 08:27:03] RAM power estimation: 3.00W for 16.00GB
[codecarbon INFO @ 08:27:03] Energy consumed for RAM : 0.000016 kWh. RAM Power : 3.0 W
[codecarbon DEBUG @ 08:27:03] Done measure for RAM - measurement time: 0.0022 s - last call 4.62 s
[codecarbon INFO @ 08:27:03] Delta energy consumed for CPU with constant : 0.000055 kWh, power : 42.5 W
[codecarbon INFO @ 08:27:03] Energy consumed for All CPU : 0.000234 kWh
[codecarbon DEBUG @ 08:27:03] Done measure for CPU - measurement time: 0.0008 s - last call 4.62 s
[codecarbon INFO @ 08:27:03] 0.000250 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 08:27:03] last_duration=4.620552458014572
------------------------
[codecarbon DEBUG @ 08:27:03] We apply an energy mix of 238 g.CO2eq/kWh for United Kingdom
[codecarbon DEBUG @ 08:27:03] EmissionsData(timestamp='2025-07-12T08:27:03', project_name='codecarbon', run_id='None', experiment_id='d960f128-b0b6-4eb2-a1e1-d2bbc81bbbe0', duration=20.052634791994933, emissi

Nodes: 4039
Edges: 88234
Average Degree: 43.69
Density: 0.010820
Longest Path: 8
Longest Path Nodes: [687, 686, 698, 3437, 567, 414, 594, 3980, 4038]
Shortest Path: 4
Shortest Path Nodes: [567, 3437, 698, 686, 855]
Execution Time: 18.22 seconds
Memory Used: -0.86 MB
Top 20 Most Connected Nodes:
  Node 107: Degree 1045
  Node 1684: Degree 792
  Node 1912: Degree 755
  Node 3437: Degree 547
  Node 0: Degree 347
  Node 2543: Degree 294
  Node 2347: Degree 291
  Node 1888: Degree 254
  Node 1800: Degree 245
  Node 1663: Degree 235
  Node 1352: Degree 234
  Node 2266: Degree 234
  Node 483: Degree 231
  Node 348: Degree 229
  Node 1730: Degree 226
  Node 1985: Degree 224
  Node 1941: Degree 223
  Node 2233: Degree 222
  Node 2142: Degree 221
  Node 1431: Degree 220


[codecarbon DEBUG @ 08:27:27] RAM power estimation: 3.00W for 16.00GB
[codecarbon INFO @ 08:27:27] Energy consumed for RAM : 0.000017 kWh. RAM Power : 3.0 W
[codecarbon DEBUG @ 08:27:27] Done measure for RAM - measurement time: 0.0020 s - last call 4.88 s
[codecarbon INFO @ 08:27:27] Delta energy consumed for CPU with constant : 0.000058 kWh, power : 42.5 W
[codecarbon INFO @ 08:27:27] Energy consumed for All CPU : 0.000237 kWh
[codecarbon DEBUG @ 08:27:27] Done measure for CPU - measurement time: 0.0008 s - last call 4.89 s
[codecarbon INFO @ 08:27:27] 0.000254 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 08:27:27] last_duration=4.883045916998526
------------------------
[codecarbon DEBUG @ 08:27:27] We apply an energy mix of 238 g.CO2eq/kWh for United Kingdom
[codecarbon DEBUG @ 08:27:27] EmissionsData(timestamp='2025-07-12T08:27:27', project_name='codecarbon', run_id='None', experiment_id='d960f128-b0b6-4eb2-a1e1-d2bbc81bbbe0', duration=20.24401837500045, emissio

Avg CPU Usage (%),▁
Avg Emissions (kgCO2),▁
Avg Memory (MB),▁
Avg Time (s),▁
CPU Usage (%),█▆▆▁▂
CPU Usage Std (%),▁
Emissions (kgCO2),▆▄▁▂█
Emissions Std (kgCO2),▁
Memory (MB),█▂▄▂▁
Memory Std (MB),▁
Run,▁▃▅▆█



Table: Performance Comparison (Averaged over runs)
                   Method  Nodes  Edges  Avg Degree  Density  Longest Path  \
0  Naive (Adjacency List)   4039  88234   43.691013  0.01082             3   
1         NetworkX (Full)   4039  88234   43.691013  0.01082             8   
2     SciPy Sparse (Full)   4039  88234   43.691013  0.01082             8   

                                 Longest Path Nodes  Shortest Path  \
0                                   [1, 0, 34, 348]              1   
1  [687, 686, 698, 3437, 567, 414, 594, 3980, 4038]              4   
2  [687, 686, 698, 3437, 567, 414, 594, 3980, 4038]              4   

          Shortest Path Nodes   Time (s)  Time Std (s)  Memory (MB)  \
0                      [0, 1]  44.391911      0.475975   -29.606250   
1  [567, 3437, 698, 686, 855]  17.785105      0.219478     2.053125   
2  [567, 3437, 698, 686, 855]  18.065151      0.144381    23.453125   

   Memory Std (MB)  Emissions (kgCO2)  Emissions Std (kgCO2)  CPU Usa